## Install dependencies and setting up GPU enviornment

In [2]:
#!pip install tensorflow==2.0.0-beta1 
#!pip install tqdm

In [3]:
!wget --no-check-certificate \
  https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip \
  -O ./cats_and_dogs_filtered.zip

--2019-08-05 20:56:24--  https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 172.217.31.144, 2404:6800:4004:800::2010
Connecting to storage.googleapis.com (storage.googleapis.com)|172.217.31.144|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 68606236 (65M) [application/zip]
Saving to: ‘./cats_and_dogs_filtered.zip’

./cats_and_dogs_fil 100%[===================>]  65.43M  69.2MB/s    in 0.9s    

2019-08-05 20:56:25 (69.2 MB/s) - ‘./cats_and_dogs_filtered.zip’ saved [68606236/68606236]



## Dataset Preprocessing

### Import project dependenceies

In [4]:
import os
import zipfile 
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

from tqdm import tqdm_notebook
from tensorflow.keras.preprocessing.image import ImageDataGenerator

%matplotlib inline
tf.__version__

'2.0.0-beta1'

### Unzipping the Dogs vs Cats dataset
  

In [0]:
dataset_path = './cats_and_dogs_filtered.zip'

In [0]:
zip_object = zipfile.ZipFile(file=dataset_path, mode='r')

In [0]:
zip_object.extractall("./")

In [0]:
zip_object.close()

### Setting up dataset paths

In [0]:
dataset_path_new = "./cats_and_dogs_filtered/"

In [0]:
train_dir = os.path.join(dataset_path_new, "train")
validation_dir = os.path.join(dataset_path_new, "validation")

## Building the model

### Loading the pre-trained model (MobileNetV2)

In [0]:
IMG_SHAPE = (128, 128, 3)